# Extracting and Visualizing Stock Data Using Yahoo Finance API and Web Scrapping

## Project Description
This project focuses on extracting and visualizing stock data for Tesla and GameStop (GME) using two methods: the Yahoo Finance API (yfinance) and web scraping. By utilizing the yfinance library, we retrieve historical stock price data for both Tesla and GME. Additionally, revenue data is collected from web scraping relevant financial websites. Finally, we visualize the stock and revenue trends to gain insights into the performance of both companies.

## Objectives
+ Extract historical stock price data for Tesla and GME using the yfinance API.
+ Use web scraping techniques to gather revenue data for Tesla and GME from external websites.
+ Visualize the extracted data, highlighting trends in stock performance and revenue over time.

<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#defining-graphing-function">Defining Graphing Function</a></li>
        <li><a href="#using-yfinance-to-extract-tesla-stock-data">Using yfinance to Extract Tesla Stock Data</a></li>
        <li><a href ="#using-web-scraping-to-extract-tesla-revenue-data
">Using Web Scraping to Extract Tesla Revenue Data</a></li>
        <li><a href="#using-yfinance-to-extract-gme-stock-data">Using yfinance to Extract GME Stock Data</a></li>
        <li><a href ="#using-web-scraping-to-extract-gme-revenue-data
">Using Web Scraping to Extract GME Revenue Data</a></li>
        <li><a href ="#plotting-tesla-stock-graph">Plotting Tesla Stock Graph</a></li>
        <li><a href ="#plotting-gme-stock-graph">Plotting GME Stock Graph</a></li>
    </ol>
</div>

<hr>


In [1]:
# Importing libraries
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Defining Graphing Function

This function is defined only for the purpose to plot the final results as line graphs.

In [2]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## Using yfinance to Extract Tesla Stock Data

Using the `Ticker` function we want to extract data on to create a ticker object. The stock is Tesla and its ticker symbol is `TSLA`.

In [3]:
tesla = yf.Ticker('TSLA')

Using the ticker object and the function `history` we extract stock information and save it in a dataframe named `tesla_data`. Setting the `period` parameter to ` "max" ` we get information for the maximum amount of time.

In [4]:
tesla_data = tesla.history(period = 'max')

**Resetting the index** using the `reset_index(inplace=True)` function on the tesla_data DataFrame and displaying the first five rows of the `tesla_data` dataframe using the `head` function.

In [5]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## Using Web Scraping to Extract Tesla Revenue Data

Using the `requests` library to download the webpage https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm and saving the text of the response as a variable named `html_data`.


In [6]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm'
html_data = requests.get(url).text

Parsing the html data using `beautiful_soup` using parser i.e `html.parser`.


In [7]:
soup = BeautifulSoup(html_data, 'html.parser')

Extracting the table with `Tesla Revenue` using `BeautifulSoup` and storing it into a dataframe named `tesla_revenue`.


In [8]:
tables = soup.find_all('table')
for index, table in enumerate(tables):
    if ('Tesla Quarterly Revenue' in str(table)):
        table_index = index

tesla_revenue = pd.DataFrame(columns = ['Date', 'Revenue'])
for row in tables[table_index].find('tbody').find_all('tr'):
    col = row.find_all('td')
    date = col[0].text.strip()
    revenue = col[1].text.strip()
    tesla_revenue = pd.concat( [tesla_revenue, pd.DataFrame({'Date':[date], 'Revenue':[revenue]})], ignore_index=True)

Removing the comma and dollar sign from the `Revenue` column. 


In [9]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"", regex=True)

<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
/var/folders/kd/x118d1610hjc9jvj9l228pxm0000gn/T/ipykernel_42034/1877950674.py:1: SyntaxWarning: invalid escape sequence '\$'
  tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"", regex=True)


Removing an null or empty strings in the Revenue column.

In [10]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

Desplaying the first 5 rows of the `tesla_revenue` dataframe using the `head` function.

In [11]:
tesla_revenue.head()

,Date,Revenue
0,2022-09-30,21454
1,2022-06-30,16934
2,2022-03-31,18756
3,2021-12-31,17719
4,2021-09-30,13757


## Using yfinance to Extract GME Stock Data

In [12]:
GameStop = yf.Ticker('GME')

gme_data = GameStop.history(period = 'max')

gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662209,1.603295,1.662209,6892800,0.0,0.0


## Using Web Scraping to Extract GME Revenue Data

In [13]:
url_gme = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html'
html_data_2 = requests.get(url_gme).text

In [14]:
soup_gme = BeautifulSoup(html_data_2, 'html.parser')

In [15]:
tables_gme = soup_gme.find_all('table')
for index, table in enumerate(tables_gme):
    if ('GameStop Quarterly Revenue' in str(table)):
        table_gme_index = index

gme_revenue = pd.DataFrame(columns= ['Date', 'Revenue'])
for row in tables_gme[table_gme_index].tbody.find_all('tr'):
    col = row.find_all('td')
    if col:
        date_gme = col[0].text.strip()
        revenue_gme = col[1].text.strip()
        gme_revenue = pd.concat([gme_revenue, pd.DataFrame({'Date':[date_gme], 'Revenue':[revenue_gme]})], ignore_index=True)

In [16]:
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"", regex=True)
gme_revenue.dropna(inplace=True)
gme_revenue = gme_revenue[gme_revenue['Revenue'] != ""]

<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
/var/folders/kd/x118d1610hjc9jvj9l228pxm0000gn/T/ipykernel_42034/3785437158.py:1: SyntaxWarning: invalid escape sequence '\$'
  gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"", regex=True)


In [17]:
gme_revenue.head()

,Date,Revenue
0,2020-04-30,1021
1,2020-01-31,2194
2,2019-10-31,1439
3,2019-07-31,1286
4,2019-04-30,1548


## Plotting Tesla Stock Graph

In this section, we visualize the historical stock price data for `Tesla (TSLA)` using a line graph. Line graphs are particularly useful for showing stock prices over time because they clearly highlight the trends, fluctuations, and patterns in the data.

In [18]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

## Plotting GME Stock Graph

Similarly, we plot the historical stock price data for `GameStop (GME)`. A line graph helps to capture the notorious volatility of `GME`'s stock prices, especially during periods of high market speculation and public interest.

In [19]:
make_graph(gme_data, gme_revenue, "GameStop")